In [18]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import copy
import matplotlib.pylab as plt 
#from mpl_toolkits.basemap import Basemap
from collections import OrderedDict


In [3]:
def insertIntoDataStruct(name,val,aDict):
    if not name in aDict:
        aDict[name] = [(val)]
    else:
        aDict[name].append((val))


In [4]:
def create_source_dictionary():
    for nfil in fil_nms:
        txt_fil = fil_dir + nfil

        with open(txt_fil) as file:
            if (nfil == 'tamakin_dacot_vertex_ROMS.txt'):
                vals = np.array([(nfil.replace('_vertex_ROMS.txt',''),seed_only_ind[int(line.split()[0])-1]) for line in file])
            else:
                next(file)
                if (nfil == 'camotes_islands_results.txt'):
                    vals = np.array([(nfil.replace('_results.txt',''),int(line.split()[0])-1) for line in file])
                else:
                    vals = np.array([(str(line.split()[0]),water_only_ind[int(line.split()[1])-1]) for line in file])
            #print(vals)

        for nst in range(vals.shape[0]):
             insertIntoDataStruct(vals[nst,0], int(vals[nst,1]), src_sink_dict)


In [5]:
def get_vel_proj(u_rho,v_rho):
    # projection
    u_plt = u_rho*(np.cos(angs)) + v_rho*(np.cos(np.pi/2 + angs))
    v_plt = u_rho*(np.sin(angs)) + v_rho*(np.sin(np.pi/2 + angs))

    return u_plt, v_plt

In [6]:
def outline_domain(ax):
    for j in range(lat_psi.shape[0]-1):
        ax.plot((lon_psi[j,0],lon_psi[j+1,0]),(lat_psi[j,0],lat_psi[j+1,0]),linewidth=3,color='k',latlon=True,zorder=map_order+5)
        ax.plot((lon_psi[j,-1],lon_psi[j+1,-1]),(lat_psi[j,-1],lat_psi[j+1,-1]),linewidth=3,color='k',latlon=True,zorder=map_order+5)
    for ii in range(lat_psi.shape[1]-1):
        ax.plot((lon_psi[0,ii],lon_psi[0,ii+1]),(lat_psi[0,ii],lat_psi[0,ii+1]),linewidth=3,color='k',latlon=True,zorder=map_order+5)
        ax.plot((lon_psi[-1,ii],lon_psi[-1,ii+1]),(lat_psi[-1,ii],lat_psi[-1,ii+1]),linewidth=3,color='k',latlon=True,zorder=map_order+5)


In [7]:
def plt_map(map_type,version_str):
    fig, ax = plt.subplots(1, 1, sharex='col', sharey='row',
                       gridspec_kw={'hspace': 0, 'wspace': 0}, 
                       figsize = (26,20))
    
    mapid = Basemap(llcrnrlon = np.min(lon_psi) - map_offset, 
              llcrnrlat = np.min(lat_psi) - map_offset, 
              urcrnrlon = np.max(lon_psi) + map_offset,
              urcrnrlat = np.max(lat_psi) + map_offset, 
              resolution = 'f',
              projection = 'merc', ax=ax)
    
    # Plot data specific to map type
    if map_type == 'vel':
        P = mapid.pcolormesh(lon_psi, lat_psi, c_spd, vmin=0,vmax=.5, cmap='YlOrRd',latlon=True,zorder=map_order+2)
        mapid.quiver(lon[1:-1:qfreq,1:-1:qfreq], lat[1:-1:qfreq,1:-1:qfreq], u_plt[::qfreq,::qfreq], v_plt[::qfreq,::qfreq], 
                 latlon=True,zorder=map_order+2)
        
        c_ticks = np.arange(0,.5+.01,.1)
        cbar_label_y = .25
        cbar_txt = 'Current Speed \n (m s$^{-1}$)'
        
        type_save_str = 'v1_mean_surface_currents_'
        
    elif map_type == 'mke':
        P = mapid.pcolormesh(lon_psi, lat_psi, 10000*mke, vmin=0,vmax=400, cmap='nipy_spectral',latlon=True,zorder=map_order+2)
        
        c_ticks = np.arange(0,400+1,100)
        cbar_label_y = 200
        cbar_txt = 'Energy Per Unit Mass \n (cm$^{2}$ s$^{-1}$)'
        
        type_save_str = 'v2_Average_Kinetic_Energy_'
        
    elif map_type == 'eke':
        P = mapid.pcolormesh(lon_psi, lat_psi, 10000*eke, vmin=0,vmax=400, cmap='nipy_spectral',latlon=True,zorder=map_order+2)
     
        c_ticks = np.arange(0,400+1,100)
        cbar_label_y = 200
        cbar_txt = 'Energy Per Unit Mass \n (cm$^{2}$ s$^{-1}$)'
        
        type_save_str = 'v2_Average_Eddy_Kinetic_Energy_'
    
    mapid.drawcoastlines(linewidth=2)
    mapid.fillcontinents(color='white')
    mapid.drawmapboundary(fill_color=(0.9,0.98,1))
    parallels = [10,11]
    # labels: [left,right,top,bottom]
    mapid.drawparallels(parallels,labels=[True,False,False,False],zorder=map_order+2,fontsize=24)
    meridians = [124,125]
    mapid.drawmeridians(meridians,labels=[False,False,True,False],zorder=map_order+2,fontsize=24)
    outline_domain(mapid)
    
    # Colorbar
    pos = ax.get_position() # get the original position 
    cax = fig.add_axes([pos.x0+pos.width+0.02*pos.width, pos.y0, 0.04*pos.width, pos.height])
    cbar = fig.colorbar(P, cax=cax, orientation='vertical')
    cax.set_yticks(c_ticks)
    cax.tick_params(labelsize=22)
    cbar.ax.text(3,cbar_label_y,cbar_txt,va='center',ha='center',fontsize=24,rotation=270)
    
    # Outline Plot and Colorbar
    plt.setp(cax.spines.values(), color='black',linewidth=2,zorder=map_order+3)
    plt.setp(ax.spines.values(), color='black',linewidth=2,zorder=map_order+3)
    
    # Save Figure
    plt.savefig((version_str + '_' + type_save_str + surf_vel_dict[fil]['sav_str']))
    

In [8]:
def data_to_csv_dict(label,data):
    data_r = data.ravel()
    for loc in src_sink_dict:
        if label == 'Location':
            #print(loc)
            insertIntoDataStruct('Location',loc,csv_dict)
        else:
            loc_vals = []
            
            for loc_idx in src_sink_dict[loc]:
                # get the values at locations
                loc_vals = np.append(loc_vals,data_r[loc_idx])
            print(loc, len(loc_vals))
            print(('mean '+ label), np.mean(loc_vals),' ± ', np.std(loc_vals))
            insertIntoDataStruct((label +'_avg'), np.mean(loc_vals), csv_dict)
            
            if len(loc_vals)>1:
                # take the standard deviation and append to dictionary
                insertIntoDataStruct((label +'_std'), np.std(loc_vals), csv_dict)
            else:
                # store NA as standard deviation value
                insertIntoDataStruct((label +'_std'), 'NA', csv_dict)
   

In [9]:
grid_fil = 'CamotesROMS_grid_file.nc'
grd_fid = nc.Dataset(grid_fil)
mask = grd_fid.variables['mask_rho'][:].squeeze()
water_only_ind = np.where(mask.ravel())[0]
h = grd_fid.variables['h'][:].squeeze()

# GENERATE mask for seeded-only points
seed_mask = np.zeros(h.shape,dtype=bool)
# MASK for coastal points and <10m depth
for jj in range(2,mask.shape[0]-1):
    for ii in range(2,mask.shape[1]-1):
        # IF POINT IS WATER
        if mask[jj,ii] == 1:
            coast = mask[jj-1,ii] + mask[jj+1,ii] + mask[jj,ii-1] + mask[jj,ii+1]
            # IF POINT TOUCHES COAST or IS LESS THAN 10m in DEPTH
            if  ((coast < 4) or (h[jj,ii]<10)) :
                seed_mask[jj,ii] = 1
                
# SEEDED_INDICES
seed_only_ind = np.where(seed_mask.ravel())[0]

# All the ROMS variables needed for plotting
lat = grd_fid.variables['lat_rho'][:]
lon = grd_fid.variables['lon_rho'][:]
mask_rho = grd_fid.variables['mask_rho'][:]
angs = grd_fid.variables['angle'][:][1:-1,1:-1]

lat_vert = grd_fid.variables['lat_vert'][:]
lon_vert = grd_fid.variables['lon_vert'][:]

lat_psi = grd_fid.variables['lat_psi'][:]
lon_psi = grd_fid.variables['lon_psi'][:]

nlat,nlon = lat.shape[:]

map_order=30
map_offset=0.01
qfreq=10

In [12]:

eke_fil_dir = './'

surf_vel_dict = {'surf_vel_2010-2015.nc':{'sav_str':'2010-2015'},
                 'surf_vel_2012-2014.nc':{'sav_str':'2012-2014'},
                 'surf_vel_Jun2011-May2012.nc':{'sav_str':'Jun2011-May2012'},
                 'surf_vel_Jun2012-May2013.nc':{'sav_str':'Jun2012-May2013'},
                 'surf_vel_Jun2013-May2014.nc':{'sav_str':'Jun2013-May2014'},
                 'surf_vel_2012-2014_NEMonsoon.nc':{'sav_str':'NEMonsoon'},
                 'surf_vel_2012-2014_SWMonsoon.nc':{'sav_str':'SWMonsoon'}}

In [13]:
fil_dir = './'
fil_nms = ['camotes_vertices_sites_results_water_only.txt', 
           'tamakin_dacot_vertex_ROMS.txt',
           'camotes_islands_results.txt']


# INITIALIZE source/sink dictionary
src_sink_dict = OrderedDict()

# POPULATE source/sink dictionary
create_source_dictionary()
   

In [19]:
for fil in surf_vel_dict:
#for fil in ['surf_vel_2010-2015.nc']:    
    print('')
    print('')
    print(fil)
    # create csv dictionary
    csv_dict = OrderedDict()
    data_to_csv_dict('Location',np.empty(1))
    
    ncfil = eke_fil_dir + fil
    fid = nc.Dataset(ncfil)
    
    u = fid.variables['u'][:].squeeze()
    v = fid.variables['v'][:].squeeze()
        
    u_rho = ((u[:,1:-1,1:] + u[:,1:-1,:-1])/2.) # drop first and last row; average columns
    v_rho = ((v[:,1:,1:-1] + v[:,:-1,1:-1])/2.) # drop first and last column; average rows
    
    # Average Current Velocity and Speed
    # Verssion 1: Time mean, interpolate, (transform), square u and v, u2+v2, square root
    avg_u = np.mean(u,axis=0)
    avg_v = np.mean(v,axis=0)
    avg_u_rho = (avg_u[1:-1,1:] + avg_u[1:-1,:-1])/2
    avg_v_rho = (avg_v[1:,1:-1] + avg_v[:-1,1:-1])/2
    u_plt,v_plt = get_vel_proj(avg_u_rho,avg_v_rho)
    c_spd = np.sqrt(avg_u_rho**2+avg_v_rho**2)
    
    #plt_map('vel', 'v1')
    data_to_csv_dict('u_(cm/s)_v1_',100*u_plt)
    data_to_csv_dict('v_(cm/s)_v1_',100*v_plt)
    data_to_csv_dict('c_(cm/s)_v1_',100*c_spd)
    
    # Version 2: interpolate to rho points, (transform North-East, time mean), square u and v, u2+v2, sqrt, time mean
    u_plt,v_plt = get_vel_proj(u_rho,v_rho) # convert vectors to North-East orientation   
    u_plt = np.mean(u_plt,axis=0) # time-average u at rho points
    v_plt = np.mean(v_plt,axis=0) # time-average v at rho points
    c_spd = np.mean(np.sqrt(u_rho**2+v_rho**2),axis=0)
    
    #plt_map('vel','v2')
    data_to_csv_dict('u_(cm/s)_v2_',100*u_plt)
    data_to_csv_dict('v_(cm/s)_v2_',100*v_plt)
    data_to_csv_dict('c_(cm/s)_v2_',100*c_spd)
    
    # AVERAGE KINETIC ENERGY
    # Version 1: time average u and v, square, interpolate to rho points, u2 + v2
    squared_avg_u = np.mean(u,axis=0)**2
    squared_avg_v = np.mean(v,axis=0)**2
    squared_avg_u_rho = (squared_avg_u[1:-1,:-1]+squared_avg_u[1:-1,1:])/2
    squared_avg_v_rho = (squared_avg_v[:-1,1:-1]+squared_avg_v[1:,1:-1])/2
    mke =(squared_avg_u_rho + squared_avg_v_rho)/2
    
    #plt_map('mke',mke,'v1')
    data_to_csv_dict('mke_(cm2/s2)_v1_',10000*mke)
    
    # Version 2: interpolate u and v to rho points, square, u2 +v2, time average
    
    mke = np.mean((u_rho**2+v_rho**2)/2,axis=0)
    
    #plt_map('mke',mke_2,'v2')
    data_to_csv_dict('mke_(cm2/s2)_v2_',10000*mke)
    
    # AVERAGE EDDY KINETIC ENERGY: STORE REFERENCES
    if fil == 'surf_vel_2010-2015.nc':
        uref = copy.deepcopy(u)
        vref = copy.deepcopy(v)
        
        uref_rho = copy.deepcopy(u_rho)
        vref_rho = copy.deepcopy(v_rho)
        print(uref_rho.shape, vref_rho.shape)
        
    else:
        # AVERAGE EDDY KINETIC ENERGY CALCULATIONS
        # Version 1: square time-mean reference u and v. subtract from time mean of u and v squared, respectively
        #            interpolate u and v to rhow points. add u and v 
        u_points_u = np.mean(u**2,axis=0) - np.mean(uref,axis=0)**2 # subtract reference
        v_points_v = np.mean(v**2,axis=0) - np.mean(vref,axis=0)**2 # subtract reference
         
        u_eke_rho = (u_points_u[1:-1,1:]+u_points_u[1:-1,:-1])/2 # interpolation to rho points
        v_eke_rho = (v_points_v[1:,1:-1]+v_points_v[:-1,1:-1])/2 # interpolation to rho points
    
        eke = (u_eke_rho + v_eke_rho)/2
        #plt_map('eke', 'v1')
        data_to_csv_dict('eke_(cm2/s2)_v1_',10000*eke)
        print('MEEP1', np.min(10000*eke),np.max(10000*eke))
        
        
        
        
        
        # Version 2: interpolate uref, vref, u and v to rho points. Square all values, (u2 + v2)/2, time mean
        eke_all = ((u_rho**2 - np.mean(uref_rho**2,axis=0)) + (v_rho**2 - np.mean(vref_rho**2,axis=0)))/2
        eke = np.mean(eke_all,axis=0)
        #print('MEEP2', np.min(10000*eke),np.max(10000*eke))
        
        # Version 2.5 (version 1 re-structured)
        #eke_all = ((u_rho**2 - np.mean(uref_rho,axis=0)**2) + (v_rho**2 - np.mean(vref_rho,axis=0)**2))/2
        #eke = np.mean(eke_all,axis=0)
        #print('MEEP3', np.min(10000*eke),np.max(10000*eke)) 
            
        # Version 2.75 (version 2 re-structured)
        #eke_all = (u_rho**2 + v_rho**2)/2 - np.mean((uref_rho**2 + vref_rho**2)/2,axis=0)
        #eke = np.mean(eke_all,axis=0)
        #print('MEEP4', np.min(10000*eke),np.max(10000*eke)) 
        
        #print(np.min(eke), np.max(eke))
        #plt_map('eke', 'v2')
        data_to_csv_dict('eke_(cm2/s2)_v2_',10000*eke)
        
        
        # Version 3: interpolate uref, vref, u and v to rho points. Square all values, (u2 + v2)/2, time mean
        #eke_all = ((u_rho - np.mean(uref_rho,axis=0))**2 + (v_rho - np.mean(vref_rho))**2)/2
        #eke = np.mean(eke_all,axis=0)
        #print('MEEP3', np.min(10000*eke),np.max(10000*eke))
        #print(np.min(eke), np.max(eke))
        #plt_map('eke', 'v3')
        #data_to_csv_dict('eke_(cm2/s2)_v3_',10000*eke)
        
    df = pd.DataFrame(csv_dict)
    df.to_csv((surf_vel_dict[fil]['sav_str']+'.csv'))
    



surf_vel_2010-2015.nc


/var/folders/7x/p_3rd8ms42zbyqxfl97h0m300000gn/T/ipykernel_16759/2845132798.py:26: RuntimeWarning: invalid value encountered in sqrt
  c_spd = np.sqrt(avg_u_rho**2+avg_v_rho**2)


Palanas 1
mean u_(cm/s)_v1_ -1.0440251379971204  ±  0.0
Wangag 1
mean u_(cm/s)_v1_ -1.2670172905249097  ±  0.0
Magbangon 1
mean u_(cm/s)_v1_ -1.3800577417294502  ±  0.0
Cabatoan 1
mean u_(cm/s)_v1_ -1.4746948208120896  ±  0.0
Caridad_Cemetery 1
mean u_(cm/s)_v1_ -2.123276836044884  ±  0.0
Caridad_Proper 1
mean u_(cm/s)_v1_ -0.6856445312618178  ±  0.0
Hicgop 1
mean u_(cm/s)_v1_ 3.2079024128502773  ±  0.0
Hicgop_South 1
mean u_(cm/s)_v1_ 3.032623436521368  ±  0.0
Sitio_Tugas 1
mean u_(cm/s)_v1_ 2.2141861436573183  ±  0.0
Elementary_School 1
mean u_(cm/s)_v1_ 1.5045476291873214  ±  0.0
Sitio_Lonas 1
mean u_(cm/s)_v1_ 1.077270331996312  ±  0.0
San_Agustin 1
mean u_(cm/s)_v1_ 0.7550156512439741  ±  0.0
Poroc_San_Flowers 1
mean u_(cm/s)_v1_ 0.4877280759254643  ±  0.0
Poroc_Rose 1
mean u_(cm/s)_v1_ 0.22748058387278766  ±  0.0
Pangasugan 1
mean u_(cm/s)_v1_ -0.15101238679786616  ±  0.0
Visca 1
mean u_(cm/s)_v1_ -0.26262783721557326  ±  0.0
Gabas 1
mean u_(cm/s)_v1_ -0.18378244003322447  ±  0.0

/Users/lizdrenkard/miniconda3/envs/mom6/lib/python3.11/site-packages/numpy/ma/core.py:6945: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/var/folders/7x/p_3rd8ms42zbyqxfl97h0m300000gn/T/ipykernel_16759/2845132798.py:37: RuntimeWarning: invalid value encountered in sqrt
  c_spd = np.mean(np.sqrt(u_rho**2+v_rho**2),axis=0)


Palanas 1
mean u_(cm/s)_v2_ -1.0440256716007321  ±  0.0
Wangag 1
mean u_(cm/s)_v2_ -1.2670163168699595  ±  0.0
Magbangon 1
mean u_(cm/s)_v2_ -1.3800571737502392  ±  0.0
Cabatoan 1
mean u_(cm/s)_v2_ -1.4746960263409985  ±  0.0
Caridad_Cemetery 1
mean u_(cm/s)_v2_ -2.123275566163827  ±  0.0
Caridad_Proper 1
mean u_(cm/s)_v2_ -0.6856431431430025  ±  0.0
Hicgop 1
mean u_(cm/s)_v2_ 3.2079027571646455  ±  0.0
Hicgop_South 1
mean u_(cm/s)_v2_ 3.032623708398444  ±  0.0
Sitio_Tugas 1
mean u_(cm/s)_v2_ 2.2141856658126406  ±  0.0
Elementary_School 1
mean u_(cm/s)_v2_ 1.5045471067705836  ±  0.0
Sitio_Lonas 1
mean u_(cm/s)_v2_ 1.0772696891192515  ±  0.0
San_Agustin 1
mean u_(cm/s)_v2_ 0.7550174254486367  ±  0.0
Poroc_San_Flowers 1
mean u_(cm/s)_v2_ 0.4877281387216497  ±  0.0
Poroc_Rose 1
mean u_(cm/s)_v2_ 0.22748035327172952  ±  0.0
Pangasugan 1
mean u_(cm/s)_v2_ -0.15101353083843938  ±  0.0
Visca 1
mean u_(cm/s)_v2_ -0.26262857880194046  ±  0.0
Gabas 1
mean u_(cm/s)_v2_ -0.18378354261083701  ±  0.

...

/var/folders/7x/p_3rd8ms42zbyqxfl97h0m300000gn/T/ipykernel_16759/2845132798.py:26: RuntimeWarning: invalid value encountered in sqrt
  c_spd = np.sqrt(avg_u_rho**2+avg_v_rho**2)


Palanas 1
mean u_(cm/s)_v1_ 1.9390427491071134  ±  0.0
Wangag 1
mean u_(cm/s)_v1_ 1.0386960800659168  ±  0.0
Magbangon 1
mean u_(cm/s)_v1_ 0.6632824999292819  ±  0.0
Cabatoan 1
mean u_(cm/s)_v1_ 0.513299400413826  ±  0.0
Caridad_Cemetery 1
mean u_(cm/s)_v1_ 1.1978593362114678  ±  0.0
Caridad_Proper 1
mean u_(cm/s)_v1_ 4.371354908704452  ±  0.0
Hicgop 1
mean u_(cm/s)_v1_ 10.414026083612567  ±  0.0
Hicgop_South 1
mean u_(cm/s)_v1_ 8.65509802602594  ±  0.0
Sitio_Tugas 1
mean u_(cm/s)_v1_ 5.342348579613346  ±  0.0
Elementary_School 1
mean u_(cm/s)_v1_ 2.580800827207223  ±  0.0
Sitio_Lonas 1
mean u_(cm/s)_v1_ 3.204804550438426  ±  0.0
San_Agustin 1
mean u_(cm/s)_v1_ 2.90055688793089  ±  0.0
Poroc_San_Flowers 1
mean u_(cm/s)_v1_ 2.538030366247827  ±  0.0
Poroc_Rose 1
mean u_(cm/s)_v1_ 2.1073467179511294  ±  0.0
Pangasugan 1
mean u_(cm/s)_v1_ 1.1627591816073362  ±  0.0
Visca 1
mean u_(cm/s)_v1_ 0.7751061843337107  ±  0.0
Gabas 1
mean u_(cm/s)_v1_ 0.29965888855265876  ±  0.0
Haina 2
mean u_(cm

/Users/lizdrenkard/miniconda3/envs/mom6/lib/python3.11/site-packages/numpy/ma/core.py:6945: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/var/folders/7x/p_3rd8ms42zbyqxfl97h0m300000gn/T/ipykernel_16759/2845132798.py:37: RuntimeWarning: invalid value encountered in sqrt
  c_spd = np.mean(np.sqrt(u_rho**2+v_rho**2),axis=0)


Palanas 1
mean u_(cm/s)_v2_ 1.939042197691575  ±  0.0
Wangag 1
mean u_(cm/s)_v2_ 1.0386958186534887  ±  0.0
Magbangon 1
mean u_(cm/s)_v2_ 0.6632821196334999  ±  0.0
Cabatoan 1
mean u_(cm/s)_v2_ 0.5132992835074041  ±  0.0
Caridad_Cemetery 1
mean u_(cm/s)_v2_ 1.1978594699430352  ±  0.0
Caridad_Proper 1
mean u_(cm/s)_v2_ 4.371354782958535  ±  0.0
Hicgop 1
mean u_(cm/s)_v2_ 10.414026969736314  ±  0.0
Hicgop_South 1
mean u_(cm/s)_v2_ 8.655097655975986  ±  0.0
Sitio_Tugas 1
mean u_(cm/s)_v2_ 5.34234986836863  ±  0.0
Elementary_School 1
mean u_(cm/s)_v2_ 2.5808010487946382  ±  0.0
Sitio_Lonas 1
mean u_(cm/s)_v2_ 3.2048050992944206  ±  0.0
San_Agustin 1
mean u_(cm/s)_v2_ 2.9005570836290606  ±  0.0
Poroc_San_Flowers 1
mean u_(cm/s)_v2_ 2.5380308472007824  ±  0.0
Poroc_Rose 1
mean u_(cm/s)_v2_ 2.107346208394742  ±  0.0
Pangasugan 1
mean u_(cm/s)_v2_ 1.162759531245387  ±  0.0
Visca 1
mean u_(cm/s)_v2_ 0.7751061088534796  ±  0.0
Gabas 1
mean u_(cm/s)_v2_ 0.2996589764588984  ±  0.0
Haina 2
mean u_(

Palanas 1
mean eke_(cm2/s2)_v1_ 23.94180379569416  ±  0.0
Wangag 1
mean eke_(cm2/s2)_v1_ 21.222835121787057  ±  0.0
Magbangon 1
mean eke_(cm2/s2)_v1_ 23.82764707129092  ±  0.0
Cabatoan 1
mean eke_(cm2/s2)_v1_ 22.952955961649934  ±  0.0
Caridad_Cemetery 1
mean eke_(cm2/s2)_v1_ 21.774777809141174  ±  0.0
Caridad_Proper 1
mean eke_(cm2/s2)_v1_ 49.06161446020598  ±  0.0
Hicgop 1
mean eke_(cm2/s2)_v1_ 183.11116091656004  ±  0.0
Hicgop_South 1
mean eke_(cm2/s2)_v1_ 138.24571027217783  ±  0.0
Sitio_Tugas 1
mean eke_(cm2/s2)_v1_ 72.70337903879525  ±  0.0
Elementary_School 1
mean eke_(cm2/s2)_v1_ 45.987224902990974  ±  0.0
Sitio_Lonas 1
mean eke_(cm2/s2)_v1_ 52.11763290814868  ±  0.0
San_Agustin 1
mean eke_(cm2/s2)_v1_ 51.4557793641418  ±  0.0
Poroc_San_Flowers 1
mean eke_(cm2/s2)_v1_ 50.88476131349016  ±  0.0
Poroc_Rose 1
mean eke_(cm2/s2)_v1_ 50.17790549687026  ±  0.0
Pangasugan 1
mean eke_(cm2/s2)_v1_ 47.8083385931286  ±  0.0
Visca 1
mean eke_(cm2/s2)_v1_ 46.624224805249675  ±  0.0
Gabas 1
m